In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [38]:
## Load the pickle files, trained model
from tensorflow.keras.models import load_model
model = load_model('regression.h5')

## Pickle files
with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_encoder_geography = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler_reg.pkl','rb') as file:
    scaler_reg = pickle.load(file)

In [39]:
# Example input data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'Exited': 0
}

In [40]:
new_df = pd.DataFrame(data = [input_data])
new_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,Male,40,3,60000,2,1,1,0


In [41]:
## Encode categorical variables
geo_encoded = onehot_encoder_geography.transform(new_df[['Geography']])
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 3)>

In [42]:
onehot_encoder_geography.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [43]:
geo = pd.DataFrame(data = geo_encoded.toarray(), columns = onehot_encoder_geography.get_feature_names_out())
new_df = pd.concat([new_df.drop(columns = 'Geography'), geo], axis = 1)
new_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,0,1.0,0.0,0.0


In [44]:
new_df['Gender'] = label_encoder_gender.transform(new_df['Gender'])
new_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,0,1.0,0.0,0.0


In [45]:
#scale the data
scale_columns = ['CreditScore','Age','Tenure','Balance','NumOfProducts']
new_df[scale_columns] = scaler_reg.transform(new_df[scale_columns])
new_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,-0.512676,1,0.088669,-0.70038,-0.260234,0.810498,1,1,0,1.0,0.0,0.0


In [46]:
# Predict Salary
prediction = model.predict(new_df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


array([[98612.875]], dtype=float32)